# Control Flows in Julia
---
* Created on 01 Sep 2023
* Created by Yooshin Oh (stevenoh0908@snu.ac.kr)
---
* <span class="mark">Documentation: https://docs.julialang.org/en/v1/manual/control-flow/</span>

## Introduction

* Julia는 Python에서 지원하는 Control Flow들은 물론, 비동기적 병렬 작업 처리를 할 수 있는 `Coroutines`와 비슷한 `Tasks`를 지원한다.
* **Julia에서 지원하는 Control Flow의 목록은 다음과 같다.**

> **Julia에서 지원하는 Control Flow들**
>
> * Compound Expresions (복합문) : `begin`, `;`
> * Conditional Evaluation (조건문, 조건 평가문) : `if-elseif-else`, `?:` (삼항 연산자)
> * Short-Circuit Evaluation : `&&`, `││`, Chained Comparisons
> * Repeated Evaluation or Loops (반복문) : `while`, `for`
> * Exception Handling (예외 처리) : `try-catch`, `error`, `throw`
> * Tasks (a.k.a. Coroutines) : `yieldto`

**`Task`를 지원하는 Julia**

* Compound Expressions, Conditional Evaluation, Short-Circuit Evaluation, Repeated Evaluation (or Loops), Exception Handling은 다른 High-Level Language에서도 지원하는 기능들이지만, `Task`는 보통 지원하지 않는 언어이다.
* Julia에서의 `Task`는 비지역적인 Control Flow를 만들어서, 잠깐씩 중지하면서 여러 `Task`로 지정된 계산 사이를 번갈아가며 계산할 수 있도록 하는 행위를 지원한다.
* Julia에서 Exception Handling과 Cooperative Multitasking은 `Tasks`를 사용하여 Julia에서 Implementation되어 있다.

## Compound Expressions (복합문)

* Julia는 `begin-end` Block이나 `;`를 이용하여 복합문을 구성할 수 있으며, 복합문을 만나면 Python처럼 차례대로 복합문 내부의 문들을 실행한 뒤, 가장 마지막 문의 평가값을 반환한다.

### `begin-end` Compound Expressions

* 복합문의 시작을 begin으로 표시, 끝을 end로 표시하고, 그 사이에 문들을 개행으로 분리하여 열거함으로써 복합문을 만들 수 있다.

In [1]:
# begin-end Compound Expressions
z = begin # start of compound expr
    x = 1
    y = 2
    x + y # this will be evaluated and returned
end

3

* `block-end` 구문으로 복합문을 구성하는 경우, Block 내부의 각 문들은 Semi-Colon(`;`)이나 개행으로 구분되어야 하지만, `begin`과 `end` 키워드는 구분될 필요가 없다.

In [3]:
begin x = 1; y = 2; x + y end

3

### Semi-Colon(`;`) Compound Expressions

* Python과 동일하게, Single 문들의 끝을 세미콜론(`;`)으로 분리하여 한 행에 여러 문들을 열거함으로써 복합문을 만들 수 있다. 복합문의 양 끝을 괄호로 감싸도 된다.

In [2]:
# semi-colon Compound Expressions
z = (x = 1; y = 2; x + y)

3

## Conditional Evaluation (조건문)

* Julia는 Bash와 유사한 Conditional Evaluation을 선보이며, `if` statement와 삼항연산자를 사용할 수 있다.

### `if-elseif-else` Construct

* Julia에서는 Python과 유사하지만, 조건문 뒤에 `:`이 없고 `if`문의 끝을 `end`로 표시하는 `if-elseif-else` Construct를 가진다.
* Python에서는 `else if`가 `elif` 키워드로 사용되었지만, Julia에서는 `elseif`이다.

In [4]:
# if-elseif-else construct example
function test(x, y)
    if x < y
        println("x is less than y")
    elseif x > y
        println("x is greater than y")
    else
        println("x is equal to y")
    end
end

test (generic function with 1 method)

In [5]:
test(1,2)

x is less than y


In [6]:
test(2,1)

x is greater than y


In [7]:
test(1,1)

x is equal to y


* Python과 유사하게, `elseif`문의 개수에는 제한이 없고, `elseif`문과 `else`문은 Optional이다.

* Python과 유사하게, `if-elseif-else` Construct는 위에서부터 조건문을 검사하면서 내려오다가 처음으로 `true`로 Evaluate된 조건 내부문을 실행하고 더 이상 그 이후 추가 조건문 검사를 하지 않고 Block의 끝으로 이동한다.

* Python과 유사하게, `if-elseif-else` Construct Block 내부에서 정의한 변수라도 '실행되었다면' 그 이후 `if` block 밖에서도 사용이 가능하다. (*`if` Blocks are "Leaky"*)

In [9]:
# if blocks are "leaky"
function test(x,y)
    if x < y
        relation = "less than"
    elseif x == y
        relation = "equal to"
    else
        relation = "greater than"
    end
    println("x is ", relation, " y.") # the variable, relation is defined in if-construct block, but available even the block of the if-construct ended.
end

test(2, 1)

x is greater than y.


* 다만 Python과 마찬가지로, `if` Block 내부에서 대입 연산이 실행되어 정의되어야만 함은 동일하다.

In [10]:
function test(x,y)
    if x < y
        relation = "less than"
    elseif x == y
        relation = "equal to"
    end
    println("x is ", relation, " y.")
end

test (generic function with 1 method)

In [11]:
test(1,2) # calls statements located in block if x < y

x is less than y.


In [12]:
test(2, 1) # Nothing Calls. There'is no case for x > y in the if-elseif construct in function test. therefore, relation is not defined.

LoadError: UndefVarError: `relation` not defined

* Julia에서는 `if`문이 **값을 반환하는데**, 조건문 실행 결과 실행되는 분기 Block 내부에서 가장 마지막 Expression을 Evaluate한 값이 반환된다. (*`if` blocks also return a value, which may seem unintuitive to users coming from many other languages. This value is simply the return value of the last executed statement in the branch that was chosen.*)

In [13]:
x = 3;
if x > 0
    "positive!"
else
    "negative"
end

"positive!"

* C, MATLAB, Perl, Python, Ruby에서는 if문에 숫자값이 들어가더라도 0 아니면 모두 true의 의미를 가졌으나, Julia는 Java처럼 if condition에 들어가는 값이 `true` 또는 `false`가 아니면 `TypeError`를 raise한다. 즉, 조건문의 조건에는 `true` 또는 `false`로 Evaluate되는 식만이 가능하다.

In [14]:
if 1  # not valid, since 1 is not among true or false
    println("true")
end

LoadError: TypeError: non-boolean (Int64) used in boolean context

### Ternary Operator (삼항 연산자)

* Julia에서는 C의 문법과 같은 방식의 **Ternary Operator (삼항 연산자)** `?:`를 쓸 수 있다. 즉: `condition ? true_val : false_val`을 사용할 수 있다.

* 다만 C와는 달리 Julia 삼항 연산자에서는 `?`와 `:` 뒤에는 공백이나 개행 문자를 반드시 주어야 한다. (구분해야 함)

In [15]:
# Example
x = 1; y = 2;
println(x < y ? "less than" : "not less than")

less than


In [16]:
# Another Example
x = 1; y = 0;
println(x < y ? "less than" : "not less than")

not less than


In [17]:
# Another Implementation for function test()
test(x,y) = println(x < y ? "x is less than y" : 
                    x > y ? "x is greater than y" : "x is equal to y") 

test (generic function with 1 method)

In [18]:
test(1,2)

x is less than y


In [19]:
test(2,1)

x is greater than y


In [20]:
test(1,1)

x is equal to y


* 삼항 연산자의 경우에도 Short-Circuit Evaluation을 한다. 즉, 굳이 Evaluate할 필요가 없는 식은 평가하지 않는다.
* 즉, `condition ? true_val : false_val`에서 `true_val`은 `condition`이 참인 경우에만 Evaluate되고, `false_val`은 `condition`이 거짓인 경우에만 Evaluate된다.

In [21]:
v(x) = (println(x); x)
1 < 2 ? v("yes") : v("no")
1 > 2 ? v("yes") : v("no")

yes
no


"no"

## Short-Circuit Evaluation

* Julia는 `&&`이나 `││`과 같이 Logical AND, OR 연산을 처리할 때는 **Short-Circuit Evaluation**을 사용한다. 즉, 이미 전체 Expression의 값이 결정되는 순간, 추가적인 Evaluation을 진행하지 않는다.
* C에서 사용했던 것처럼, Short-Circuit Evaluation은 제1항의 True/False 값에 따라서 제2항을 평가하거나 하지 않을 수 있으므로, Conditional Statement처럼 활용할 수 있다.

**Examples**

In [22]:
t(x) = (println(x); true) # will evaluated as true, when calling t(x)
f(x) = (println(x); false) # will evaluated as false, when calling f(x)

f (generic function with 1 method)

In [23]:
t(1) && t(2) # t(1) -> evaluated as 'true', therefore t(2) will be evaluated.

1
2


true

In [24]:
t(1) && f(2) # t(1) -> evaluated as 'true', therefore f(2) will be evaluated.

1
2


false

In [25]:
f(1) && t(2) # f(1) -> evaluated as 'false', therefore t(2) will not be evaluated.

1


false

In [26]:
f(1) && f(2) # f(1) -> evaluated as 'false', therefore f(2) will not be evaluated.

1


false

In [27]:
t(1) || t(2) # t(1) -> evaluated as 'true', therefore t(2) will not be evaluated.

1


true

In [28]:
t(1) || f(2) # t(1) -> evaluated as 'true', therefore f(2) will not be evaluated.

1


true

In [30]:
f(1) || t(2) # f(1) -> evaluated as 'false', therefore t(2) will be evaluated.

1
2


true

In [31]:
f(1) || f(2) # f(1) -> evaluated as 'false', therefore f(2) will be evaluated.

1
2


false

* C 등에서 `if`-statement를 줄이는 것처럼, 이 기능을 Julia에서 조건문을 줄여 쓰는 방식으로 활용할 수 있다.
* Short-Circuit Evaluation 덕분에 다음과 같이 줄여 쓸 수 있다.
    * `<cond> <statement>` -> `<cond> && <statement>`
    * `! <cond> <statement>` -> `<cond> || <statement>`

In [40]:
function fact(n::Int)
    n >= 0 || error("n must be non-negative") # <=> if !(n >= 0); error(...); end
    n == 0 && return 1 # <=> if (n == 0); return 1; end
    n * fact(n-1) # <=> return n * fact(n-1)
end

fact (generic function with 1 method)

In [33]:
fact(5)

120

In [34]:
fact(0)

1

In [35]:
fact(-1)

LoadError: n must be non-negative

In [41]:
# Short-Circuit Evaluation을 이용하여 Conditional Init가 가능하다.
true && (x = (1, 2, 3))

(1, 2, 3)

In [42]:
false && (x = (1, 2, 3)) # x will not be defined

false

* **주의**: 논리 연산이 아니라, Bitwise 연산인 `&`와 `│`의 경우는 Short-Circuit Evaluation이 적용되지 않음에 주의.

In [37]:
f(1) & t(2) # print 1, print 2, true & false -> false

1
2


false

In [38]:
t(1) | t(2) # print 1, print 2, true | true -> true

1
2


true

* **주의**: `if`, `elseif` 등과 같이, `&&`, `││`는 **두 Boolean Type에 대한 이항연산**이므로, 양 항이 Boolean Type이 아니면 `TypeError`가 raise된다. 즉, Julia에서는 C에서와 같이 임의의 정수, 실수형 자료형이 0인지의 여부로 T/F값을 가지지 않기 때문에 논리 연산에서 주의해야 한다.

In [39]:
1 && true # invalid operation, since 1 is not boolean type

LoadError: TypeError: non-boolean (Int64) used in boolean context

## Repeated Evaluation: Loops (반복문)

* Python과 유사하게, Julia에서는 다음의 두 Loop문을 지원한다.
    1. `while`문
    2. `for`문

### `while`문

* Julia에서의 `while`문은 while-Block을 `:`와 Indentation으로 구분하는 것이 아니라, `while cond` ~ `end`로 구분하는 것 이외에는 Python과 정확히 동일하다.

In [43]:
i = 1
while i <= 3
    println(i)
    global i += 1 # increase i, in global scope, as 1
end

1
2
3


### `for`문

### `for`문 (Without `in`)

* Julia에서의 `for`문은 for-Block을 `end`를 사용하여 구분하는 것 제외하고는 MatLAB의 `for`문 문법과 유사하다.

In [44]:
# Examples for for-construct
for i = 1:3 # 1:3 -> a range object. (MatLAB-like syntax)
    println(i)
end

1
2
3


* Python의 경우 `for ~ in`문으로 Iteratable Object를 넘겨서 순회할 수 있었지만, Julia는 Matlab-like 문법을 채택하고 있어, `in_loop_var = range_obj`의 형태로 for construct를 구성해야 한다.
* Julia for-loop의 동작 방식은 MatLAB의 for loop의 동작 방식과 거의 같다.

* Julia에서 `for`문의 In-for variable은 `for`문 내부에서만 Available하고, 이전에 정의되었다 하더라도 new-scope에서 새롭게 정의되어 사용된다. (즉, for문 이전에 같은 이름으로 변수가 정의되어 있다 하더라도, 새로운 공간에 변수를 잡은 뒤 상위 Identifier를 for문 안에서 숨김 처리한다.)

In [45]:
for j = 1:3
    println(j)
end

1
2
3


In [46]:
j # j is not previously defined, therefore it is known only in the for-construct.

LoadError: UndefVarError: `j` not defined

In [47]:
j = 0;
for j = 1:3
    println(j)
end
j # j, defined in for-construct, is a newly assigned variable, therefore j, calling in here is outer-scope variable j.

1
2
3


0

### `for`문 (With `in`)

* Julia에서는 Python과 유사하게 `for ~ in`으로 Container Type 자료형에 대해 순회할 수 있다. 이 때, `in` 키워드 대신에 `=`나 `∈` (Julia REPL에서 `\in` + `Tab`으로 입력 가능함) 로 대체할 수도 있다.

In [48]:
for i in [1, 4, 0] # 1-dimensional vector is iteratable
    println(i)
end

1
4
0


In [50]:
for s = ["foo", "bar", "baz"] # = can be used instead of 'in' keyword.
    println(s)
end

foo
bar
baz


In [51]:
for s ∈ ["foo", "bar", "baz"] # ∈ can be used instead of 'in'.
    println(s)
end

foo
bar
baz


### `break` & `continue`

* Python과 마찬가지로 Julia에서도 Loop문 내부에서 `break`와 `continue`를 사용할 수 있으며, 이들의 기능은 Python에서와 정확히 같다.

In [52]:
i = 1
while true
    println(i)
    if i >= 3
        break
    end
    global i += 1
end

1
2
3


In [53]:
for j = 1:1000
    println(j)
    if j >= 3
        break
    end
end

1
2
3


In [54]:
for i = 1:10
    if i % 3 != 0
        continue
    end
    println(i)
end

3
6
9
